# Easy Chat Bot
Let's import some useful tools

In [1]:
import json
from collections import defaultdict
import string
import numpy as np
from tensorflow import keras
from random import randint

The data is stored in 'intents.json' file that I'm not sure where I have found

In [2]:
with open('intents.json') as fp:
    contents = json.loads(fp.read())['intents']

Let's create a function for parsing a string into separate words

In [3]:
translator = str.maketrans('','',string.punctuation + '0123456789')

# function for parsing. We remove punctuation and numbers, map to lowercase and split at spaces
def parse(sentence):
    return sentence.translate(translator).lower().split()

In [4]:
unique_words = set() # we'll need a set to find every word occuring in the training data

We'll prepare the training data

In [5]:
tokenized_sentences = [] # arrays of tokenized sentences for each class of input text
class_responses = [] # responses for each class of input text

for i, text_data in enumerate(contents):
    name = text_data['tag']
    tokenized_sentences.append([])
    class_responses.append([])
    
    for sentence in text_data['patterns']:
        tok_sentence = parse(sentence)
        tokenized_sentences[i].append(tok_sentence)
        
        for word in tok_sentence:
            unique_words.add(word)
    
    for sentence in text_data['responses']:
        class_responses[i].append(sentence)

To create a bag of words vector we need to assign an index for each word occuring in training input data

In [6]:
unique_word_list = sorted(unique_words)
indexing = defaultdict(lambda: -1)
for i, word in enumerate(unique_word_list):
    indexing[word] = i

In [7]:
# function for bag of words encoding
def encode_bow(string_list,indexing,unique):
    vector = np.zeros(len(indexing.keys()))
    for word in string_list:
        if word in unique:
            vector[indexing[word]] += 1
    return vector

Now we only need to vectorize the sentences ...

In [8]:
sentences_encoded = []
X,Y = [],[]

for i, sentences in enumerate(tokenized_sentences):
    y_vector = np.zeros(len(class_responses))
    y_vector[i] = 1
    for sentence in sentences:
        X.append(encode_bow(sentence,indexing,unique_words))
        Y.append(y_vector)
        
order = np.random.permutation(len(X))

X,Y = np.array(X)[order],np.array(Y)[order]

... initialize a model ...

In [9]:
model = keras.Sequential()
model.add(keras.layers.Dense(200, input_shape=(len(X[0]),), activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(len(Y[0]), activation='softmax'))

optimizer = keras.optimizers.SGD(momentum=0.6,decay=10**-6,nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

... and train it

In [10]:
model.fit(X, Y, epochs=300, batch_size=5) #  ,verbose=1??

Epoch 1/300
10/10 [==============================] - 0s 2ms/step - loss: 2.6601 - accuracy: 0.0638
Epoch 2/300
10/10 [==============================] - 0s 2ms/step - loss: 2.5840 - accuracy: 0.1915
Epoch 3/300
10/10 [==============================] - 0s 2ms/step - loss: 2.5616 - accuracy: 0.0851
Epoch 4/300
10/10 [==============================] - 0s 2ms/step - loss: 2.4380 - accuracy: 0.2766
Epoch 5/300
10/10 [==============================] - 0s 1ms/step - loss: 2.4315 - accuracy: 0.1702
Epoch 6/300
10/10 [==============================] - 0s 1ms/step - loss: 2.4000 - accuracy: 0.1489
Epoch 7/300
10/10 [==============================] - 0s 1ms/step - loss: 2.3296 - accuracy: 0.2979
Epoch 8/300
10/10 [==============================] - 0s 1ms/step - loss: 2.2796 - accuracy: 0.3617
Epoch 9/300
10/10 [==============================] - 0s 1ms/step - loss: 2.2391 - accuracy: 0.3191
Epoch 10/300
10/10 [==============================] - 0s 1ms/step - loss: 2.2204 - accuracy: 0.3404
Epoch 11/

Epoch 83/300
10/10 [==============================] - 0s 2ms/step - loss: 0.1792 - accuracy: 1.0000
Epoch 84/300
10/10 [==============================] - 0s 2ms/step - loss: 0.2228 - accuracy: 1.0000
Epoch 85/300
10/10 [==============================] - 0s 2ms/step - loss: 0.2026 - accuracy: 0.9574
Epoch 86/300
10/10 [==============================] - 0s 2ms/step - loss: 0.2310 - accuracy: 0.9574
Epoch 87/300
10/10 [==============================] - 0s 2ms/step - loss: 0.2814 - accuracy: 0.9362
Epoch 88/300
10/10 [==============================] - 0s 2ms/step - loss: 0.2230 - accuracy: 0.9574
Epoch 89/300
10/10 [==============================] - 0s 2ms/step - loss: 0.1761 - accuracy: 1.0000
Epoch 90/300
10/10 [==============================] - 0s 2ms/step - loss: 0.1824 - accuracy: 0.9574
Epoch 91/300
10/10 [==============================] - 0s 2ms/step - loss: 0.2697 - accuracy: 0.9362
Epoch 92/300
10/10 [==============================] - 0s 2ms/step - loss: 0.1490 - accuracy: 0.9787


10/10 [==============================] - 0s 2ms/step - loss: 0.0625 - accuracy: 1.0000
Epoch 165/300
10/10 [==============================] - 0s 2ms/step - loss: 0.1031 - accuracy: 0.9787
Epoch 166/300
10/10 [==============================] - 0s 2ms/step - loss: 0.0604 - accuracy: 1.0000
Epoch 167/300
10/10 [==============================] - 0s 2ms/step - loss: 0.0664 - accuracy: 1.0000
Epoch 168/300
10/10 [==============================] - 0s 2ms/step - loss: 0.0597 - accuracy: 1.0000
Epoch 169/300
10/10 [==============================] - 0s 2ms/step - loss: 0.0704 - accuracy: 1.0000
Epoch 170/300
10/10 [==============================] - 0s 2ms/step - loss: 0.0703 - accuracy: 1.0000
Epoch 171/300
10/10 [==============================] - 0s 2ms/step - loss: 0.0563 - accuracy: 1.0000
Epoch 172/300
10/10 [==============================] - 0s 2ms/step - loss: 0.0489 - accuracy: 1.0000
Epoch 173/300
10/10 [==============================] - 0s 2ms/step - loss: 0.0610 - accuracy: 1.0000
Epoc

10/10 [==============================] - 0s 2ms/step - loss: 0.0405 - accuracy: 1.0000
Epoch 246/300
10/10 [==============================] - 0s 2ms/step - loss: 0.0487 - accuracy: 0.9787
Epoch 247/300
10/10 [==============================] - 0s 2ms/step - loss: 0.0322 - accuracy: 1.0000
Epoch 248/300
10/10 [==============================] - 0s 2ms/step - loss: 0.0143 - accuracy: 1.0000
Epoch 249/300
10/10 [==============================] - 0s 2ms/step - loss: 0.0381 - accuracy: 1.0000
Epoch 250/300
10/10 [==============================] - 0s 2ms/step - loss: 0.0390 - accuracy: 1.0000
Epoch 251/300
10/10 [==============================] - 0s 2ms/step - loss: 0.0588 - accuracy: 0.9787
Epoch 252/300
10/10 [==============================] - 0s 2ms/step - loss: 0.0222 - accuracy: 1.0000
Epoch 253/300
10/10 [==============================] - 0s 2ms/step - loss: 0.0152 - accuracy: 1.0000
Epoch 254/300
10/10 [==============================] - 0s 2ms/step - loss: 0.0681 - accuracy: 0.9787
Epoc

### Output
To choose a response we first need to categorize user's input. Our model will classify the input text

In [11]:
def class_predict(sentence, model, indexing, unique_words):
    vector = encode_bow(parse(sentence),indexing, unique_words)
    prediction = model.predict(vector.reshape(1,-1)).squeeze()
    if prediction.max() < 0.2:
        return None
    else: return prediction.argmax()

We'll be choosing a random sentence from responses for each input class

In [12]:
def get_response(class_index, class_responses):
    responses = class_responses[class_index]
    n = len(responses)
    return responses[randint(0,n-1)]

Let's see how it works

In [13]:
v = class_predict("hello there",model,indexing,unique_words)
print(get_response(v,class_responses))

1/1 [==============================] - 0s 84ms/step
Hello, anything I can do?


In [14]:
v2 = class_predict("can you help me out please?",model,indexing,unique_words)
print(get_response(v2,class_responses))


1/1 [==============================] - 0s 21ms/step
I can guide you through Adverse drug reaction list, Blood pressure tracking, Hospitals and Pharmacies


In [15]:
v3 = class_predict("Okay thank you I'll need to develop you further in the future",model,indexing,unique_words)
print(get_response(v3,class_responses))

1/1 [==============================] - 0s 20ms/step
Happy to help!


Unsurprisingly such small dataset won't allow us to create any complex chatbot. Moreover the bag of words methode isn't perfect(for example it doesn't allow us to include such important information as word order). I'll need to get better knowledge on natural language processing to develop this project further